In [11]:
import pandas as pd
import pandas_profiling as pdp
import numpy as np
import seaborn as sns
import matplotlib
import requests
import time
from bs4 import BeautifulSoup
from tqdm import tqdm
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline

In [12]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

train = train.rename(columns={'賃料': 'target', '契約期間': 'Contract', '間取り': 'Room',
                                                      '築年数': 'Passed', '駐車場': 'Parking', '室内設備': 'Facility',
                                                      '放送・通信': 'Internet', '周辺環境': 'Building', '建物構造': 'Material',
                                                      '面積': 'Area', 'キッチン': 'Kitchen', '所在地': 'Place',
                                                      'バス・トイレ': 'Bath', '所在階': 'Floor', 'アクセス': 'Access',
                                                      '方角': 'Angle'})

test = test.rename(columns={'契約期間': 'Contract', '間取り': 'Room', '築年数': 'Passed',
                                                    '駐車場': 'Parking', '室内設備': 'Facility', '放送・通信': 'Internet',
                                                    '周辺環境': 'Building', '建物構造': 'Material', '面積': 'Area',
                                                    'キッチン': 'Kitchen', '所在地': 'Place', 'バス・トイレ': 'Bath',
                                                    '所在階': 'Floor', 'アクセス': 'Access', '方角': 'Angle'})

In [54]:
train["Place"] = train["Place"].str.replace("東京都", "").str.split("区", expand=True)[0]
train["Area"] = train["Area"].str.replace("m2", "").astype(float).round()
train["Passed"] = train["Passed"].str.replace("新築", "0年").str.split("年", expand=True)[0].astype(int)

In [55]:
# 区ごとに築年数と面積の平均を特徴量に追加
passed_mean = train.groupby("Place")["Passed"].mean()
area_mean = train.groupby("Place")["Area"].mean()
train["Passed_mean_per_ward"] = train["Place"].map(passed_mean)
train["Area_mean_per_ward"] = train["Place"].map(area_mean)

In [56]:
# Roomについて
train["N_room"] = train["Room"].str[0].astype(int)  # strの１文字目（部屋数）を取得
train["Area_per_nroom"] = train["Area"].round() / train["N_room"]
temp_room = train["Room"].str[1:].replace("R", "")
train["L_room"] = temp_room.str.contains("L") * 1
train["D_room"] = temp_room.str.contains("D") * 1
train["K_room"] = temp_room.str.contains("K") * 1
train["S_room"] = temp_room.str.contains("S") * 1
train["N_room2"] = train["N_room"] + train["L_room"] + train["D_room"] + train["K_room"] + train["S_room"]

In [20]:
# アクセス
one_access = train["Access"].str.split("分", expand=True)[0]
one_access = one_access.str.replace("エクスプレス", "エクスプレス線")
one_access = one_access.str.replace("ライン", "ライン線")
one_access = one_access.str.replace("ライナー", "ライナー線")
one_access = one_access.str.replace("ゆりかもめ", "ゆりかもめ線")
one_access = one_access.str.split("線", expand=True)
one_access[1] = one_access[1].str.split("駅", expand=True)[0].str.replace("\t", "")
train["First_line"] = one_access[0]
train["First_line"].unique()

array(['都営三田', '都営大江戸', '京王', '総武', '京成金町', '常磐', '西武池袋', '東急東横', '南北',
       '埼京', '都営浅草', '京成本', '銀座', '中央', '京王新', '東武東上', '都営新宿', '半蔵門',
       '西武新宿', '有楽町', '京浜東北', 'つくばエクスプレス', '小田急小田原', '副都心', '東西', '東急大井町',
       '山手', '東急池上', '日比谷', '京成押上', '東武スカイツリーライン', '東急田園都市', '京急本',
       '東急目黒', '千代田', '丸ノ内', '日暮里・舎人ライナー', '京王井の頭', '東武亀戸', '東急多摩川',
       'ゆりかもめ', '都電荒川', '東急世田谷', '北総', '総武本', '東海道新幹', '京葉', '高崎', '西武豊島',
       '京急空港', '横須賀', 'りんかい', '西武有楽町', '東武大師', '東武伊勢崎', '東北本',
       '東京モノレール羽田', '東海道本', '湘南新宿ライン', '東北新幹', '京成成田空港'], dtype=object)

In [25]:
# アクセス
one_access = test["Access"].str.split("分", expand=True)[0]
one_access = one_access.str.replace("エクスプレス", "エクスプレス線")
one_access = one_access.str.replace("ライン", "ライン線")
one_access = one_access.str.replace("ライナー", "ライナー線")
one_access = one_access.str.replace("ゆりかもめ", "ゆりかもめ線")
one_access = one_access.str.split("線", expand=True)
one_access[1] = one_access[1].str.split("駅", expand=True)[0].str.replace("\t", "")


,0,1,2,3
28872,中央本,(東京－松本)新宿,None,None
29428,中央本,(東京－松本)新宿,None,None


In [37]:
one_access[one_access[0] == "埼玉高速鉄道"]

,0,1,2,3
31156,埼玉高速鉄道,赤羽岩淵,None,None


In [38]:
test[31156:31157]

,id,Place,Access,Room,Passed,Angle,Area,Floor,Bath,Kitchen,Internet,Facility,Parking,Building,Material,Contract
31156,62627,東京都北区志茂５丁目,埼玉高速鉄道線\t赤羽岩淵駅\t徒歩8分\t\t南北線\t志茂駅\t徒歩10分\t\t京浜東...,1R,1年11ヶ月,NaN,7.25m2,1階／2階建,共同トイレ／\tシャワー／\t温水洗浄便座／\t洗面台独立,給湯,NaN,エアコン付\t室内洗濯機置場\t公営水道／\t都市ガス,駐車場\t無,NaN,木造,2年間\t※この物件は\t定期借家\tです。


In [10]:
train_new["First_line"].unique()

array(['都営三田', '都営大江戸', '京王', '総武', '京成金町', '常磐', '西武池袋', '東急東横', '南北',
       '埼京', '都営浅草', '京成本', '銀座', '中央', '京王新', '東武東上', '都営新宿', '半蔵門',
       '西武新宿', '有楽町', '京浜東北', 'つくばエクスプレス', '小田急小田原', '副都心', '東西', '東急大井町',
       '山手', '東急池上', '日比谷', '京成押上', '東武スカイツリーライン', '東急田園都市', '京急本',
       '東急目黒', '千代田', '丸ノ内', '日暮里・舎人ライナー', '京王井の頭', '東武亀戸', '東急多摩川',
       'ゆりかもめ', '都電荒川', '東急世田谷', '北総', '総武本', '東海道新幹', '京葉', '高崎', '西武豊島',
       '京急空港', '横須賀', 'りんかい', '西武有楽町', '東武大師', '東武伊勢崎', '東北本',
       '東京モノレール羽田', '東海道本', '湘南新宿ライン', '東北新幹', '京成成田空港'], dtype=object)